In [6]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from tqdm import tqdm

In [7]:
#---------------------------------------------------------------------------
#target : find the list of the names from wikipedia, start from letter A by pressing next page iterate.
correct_names = []#list of names.
url =f"https://en.wikipedia.org/wiki/Category:Surnames?from=A"
while(True):#run till we get False of next page.
  next_page = False#our flag
  response = requests.get(url)
  soup = BeautifulSoup(response.content,"html.parser")
  bodyDiv = soup.find("div",{"id":"mw-pages"})
  namesDiv = bodyDiv.find("div",{"class":"mw-category mw-category-columns"})
  names = namesDiv.findAll("a")#save the names for the iteration.
  for name in names:#make our list correct.
    correct_names.append(name.text)
  bodyDiv = soup.find("div",{"id":"mw-pages"})
  names = bodyDiv.findAll("a")
  for name_link in names:#run through the links of the names till we get FALSE from flag.
    if (name_link.text == 'next page'):
      next_page = True
      url = "https://en.wikipedia.org" + name_link['href']#if there is no false then start the next page.
  if next_page == False:
    break

In [8]:

#---------------------------------------------------------------------------
Qֹֹֹ_and_name = []#name that has wikidata value
page_ids = []#list of Q_ID for wikiData
#---------------------------------------------------------------------------
page_titles = list(correct_name for correct_name in correct_names )
for page_title in tqdm(page_titles):
  api_url = f'https://en.wikipedia.org/w/api.php?action=query&titles={page_title}&prop=pageprops&format=json'
  response = requests.get(api_url).json()
  try:#if the name has id then we save it or else we pass.
    page_id = next(iter(response['query']['pages'].values()))['pageprops']['wikibase_item']
    page_ids.append(page_id)
    Qֹֹֹ_and_name.append(page_title)
  except:
    pass

100%|██████████| 94521/94521 [2:29:17<00:00, 10.55it/s]


In [12]:
len(correct_names)

94511

In [9]:
def func(page_ids,x,name_index):
  #------------------------------------------------------------------------------
  table = []#the final list of the rows
  # name_index = 0#index for running through the name list
  #------------------------------------------------------------------------------
  #here we make the final list with rows for every Q_number
  # page_ids = ['Q21502297']
  list_ex1 = []
  for Q_number in tqdm(page_ids):
    try:
      url = f"https://www.wikidata.org/wiki/{Q_number}"
      response = requests.get(url)
      soup = BeautifulSoup(response.content,"html.parser")
      enteries = soup.findAll("div",{"class":"mw-collapsible-content"})
      li_list = enteries[0].findAll("li")
      count = 0
      # print(li_list)
      new_span_c = []
      for li in li_list :#creat the rows of the table
        span_site_link = li.findAll("span",{"class":"wikibase-sitelinkview-page"})
        span = soup.findAll("span",{"class":"wikibase-descriptionview-text"})
        container = enteries[0].findAll("span",{"class":"wikibase-sitelinkview-siteid-container"})
        if len(new_span_c) == 0:
          for i in range(len(container)):
            span_continer = container[i].findAll("span")
            new_span_c.append(span_continer[0])
        table.append([Qֹֹֹ_and_name[name_index],Q_number,span[0].getText(),new_span_c[0].get("title"),span_site_link[0].get("lang"),span_site_link[0].getText()])
        new_span_c.pop(0)
    except:
      print(Q_number)
      list_ex1.append(Q_number)
    name_index+=1

  #------------------------------------------------------------------------------
  #creat the table
  results_df = pd.DataFrame(table, columns=['Label','WikiDate ID', 'English Description','Language','Wiki Short Lang','Entry'])
  # results_df = results_df[['Label','WikiDate ID', 'English Description','Language','Wiki Short Lang','Entry']]#change the coulmns
  results_df#print, it is our test 
  results_df.to_csv('Surnames'+x+'.csv', index=False, encoding='utf-8-sig')
  return name_index
  # conc.append(results_df)

In [14]:
len(page_ids)

70691

In [15]:
name_index = 0

In [ ]:
index = func(page_ids[:10000],'1',name_index)
print(index)

In [ ]:
index = func(page_ids[10000:20000],'2',index)
print(index)

In [ ]:
index = func(page_ids[20000:30000],'3',index)
print(index)

In [ ]:
index = func(page_ids[30000:40000],'4',index)
print(index)

In [ ]:
index = func(page_ids[40000:50000],'5',index)
print(index)

In [ ]:
index = func(page_ids[50000:60000],'6',index)
print(index)

In [ ]:
index = func(page_ids[60000:70000],'7',index)
print(index)

In [ ]:
index = func(page_ids[70000:],'8',index)
print(index)

In [ ]:
# We had to split the run into 8 parts because
#  the number of names with Q_ids is 70691 and  Google Colab
#  cannot handle a long hour run.

In [14]:
t1 = pd.read_csv('Surnames1.csv')#0-1
t2 = pd.read_csv('Surnames2.csv')#1-2
t3 = pd.read_csv('Surnames3.csv')#2-3
t4 = pd.read_csv('Surnames4.csv')#3-4
t5 = pd.read_csv('Surnames5.csv')#4-5
t6 = pd.read_csv('Surnames6.csv')#5-6
t7 = pd.read_csv('Surnames7.csv')#6-7
t8 = pd.read_csv('Surnames8.csv')#7-...



big_table = [t1,t2,t3,t4,t5,t6,t7,t8]
big_table_df = pd.concat(big_table,ignore_index = True)
#we use concat to create the big table


In [15]:
big_table_df

,Label,WikiDate ID,English Description,Language,Wiki Short Lang,Entry
0,Aa (surname),Q16479291,family name,English,en,Aa (surname)
1,Aa (surname),Q16479291,family name,Estonian,et,Aa (perekonnanimi)
2,Aa (surname),Q16479291,family name,Burmese,my,အား (မျိုးရိုး)
3,Aa (surname),Q16479291,family name,Turkish,tr,Aa (soyadı)
4,Aa (surname),Q16479291,family name,Vietnamese,vi,Aa (họ)
...,...,...,...,...,...,...
126601,Żywiecki (surname),Q18385785,family name,English,en,Żywiecki (surname)
126602,Żywiecki (surname),Q18385785,family name,Chinese (Min Nan),NaN,Żywiecki (sèⁿ)
126603,Zziwa,Q109540864,family name,English,en,Zziwa
126604,Zziwa,Q109540864,family name,Igbo,ig,Zziwa


In [16]:
big_table_df.to_csv('finalSurnames.csv', index=False, encoding='utf-8-sig')
